In [1]:
import zipfile
import os
from io import BytesIO
from datetime import datetime

import polars as pl
import pandas as pd

pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', None)

In [2]:
BASE_URL = os.environ.get("BASE_URL")
print(BASE_URL)

/home/prueba/cgr_corrupcion/


In [3]:
# datasets
folder_data = BASE_URL + 'data/'

datasets = {
    'ds_airhsp' : 'DS01_AIRHSP.zip',
    'ds_seace' : 'DS02_SEACE.zip',
    'ds_siaf' : 'DS03_ORDENES_SIAF.zip',
    'ds_empresas' : 'DS04_REPRESENTANTES_EMPRESAS.zip',
    'ds_servir' : 'DS05_SANCIONADOS_SERVIR.zip',
    'ds_gstos' : 'DS06_GASTOS_EJECUTORA.zip',
    'ds_proveedor' : 'DS07_PROVEEDORES_PROYECTOS.zip',
    'ds_proyectos' : 'DS07_PROYECTOS_INVERSION.zip',
    'ds_inco' : 'DS08_INCO.zip'
}

In [4]:
def get_dataset_multiple_files(id_dataset):

    archivo_zip = datasets[id_dataset]
    ruta_zip = os.path.join(folder_data, archivo_zip)
    
    # Crear un DataFrame vacío
    df = pd.DataFrame()
    
    with zipfile.ZipFile(ruta_zip, 'r') as zip_ref:
        # Iterar sobre los nombres de los archivos en el zip
        for nombre_archivo in zip_ref.namelist():
            # Verificar si el archivo es un archivo CSV
            if nombre_archivo.endswith('.csv'):
                # Leer el contenido del archivo CSV
                with zip_ref.open(nombre_archivo) as archivo_csv:
                    # contenido_csv = archivo_csv.read().decode('utf-8')
                    
                    # # Utilizar BytesIO para convertir el contenido en un objeto de bytes
                    # bytes_io = BytesIO(contenido_csv)
                    
                    # Leer el DataFrame desde el objeto de bytes
                    df_temp = pd.read_csv(archivo_csv)
                    
                    # Agregar el DataFrame al DataFrame principal
                    df = pd.concat([df, df_temp])
    
    # Imprimir el DataFrame resultante
    return df

#### SERVIR

In [5]:
%%time

servir_data = get_dataset_multiple_files("ds_servir")

CPU times: user 129 ms, sys: 26.1 ms, total: 155 ms
Wall time: 157 ms


In [6]:
servir_data.head()

,NTDI_CODIGO,CTDI_DESCRIPCION,CSAS_PER_NRODOC,CENT_RUC,CENT_NOMBRE,OBANT_NIVEL_GOB,OBANT_SECTOR,NOMBRE_DEPARTAMENTO,NOMBRE_PROVINCIA,NOMBRE_DISTRITO,CSAS_CATEGORIA,CSAS_TIPO,CSAS_CAUSA,DSAS_FECINI,DSAS_FECFIN,CSAS_INHABILITA
0,1,DNI,10004780,20393274655,UNIDAD DE GESTIÓN EDUCATIVA LOCAL CORONEL PORT...,GOBIERNO REGIONAL,GOBIERNOS REGIONALES,UCAYALI,CORONEL PORTILLO,CALLARIA,INHABILITACIÓN DEL PODER JUDICIAL,"PRIVACIÓN DE LA FUNCIÓN, CARGO O COMISIÓN QUE ...",DELITOS CONTRA LA LIBERTAD,06/06/2018,05/06/2023,SI
1,1,DNI,10039068,20195970751,DIRECCIÓN REGIONAL DE EDUCACIÓN UCAYALI,GOBIERNO REGIONAL,GOBIERNOS REGIONALES,UCAYALI,CORONEL PORTILLO,CALLARIA,ADMINISTRATIVA,DESTITUCION - LEY 29988,DELITO DE TRÁFICO ILÍCITO DE DROGAS,02/06/2018,NaN,SI
2,1,DNI,10106861,20533025057,UNIDAD DE GESTION EDUCATIVA LOCAL TACNA - UGEL...,GOBIERNO REGIONAL,GOBIERNOS REGIONALES,TACNA,TACNA,TACNA,DISCIPLINARIA,CESE TEMPORAL,TRANSGRESIÓN DE LOS DEBERES DE LA FUNCIÓN DOCENTE,28/01/2022,28/03/2022,SI
3,1,DNI,10124840,20170639520,PROYECTO ESPECIAL AFIANZAMIENTO Y AMPLIACION D...,GOBIERNO REGIONAL,GOBIERNOS REGIONALES,TACNA,TACNA,TACNA,DISCIPLINARIA,SUSPENSIÓN,ACTUAR O INFLUIR EN OTROS SERVIDORES PARA OBTE...,09/04/2022,08/04/2023,NO
4,1,DNI,10188673,20404224809,DIRECCIÓN REGIONAL DE EDUCACIÓN SAN MARTÍN - M...,GOBIERNO REGIONAL,GOBIERNOS REGIONALES,SAN MARTIN,MOYOBAMBA,MOYOBAMBA,ADMINISTRATIVA,INHABILITACION DEFINITIVA - LEY 29988,DELITOS DE VIOLACIÓN DE LA LIBERTAD SEXUAL E I...,06/06/2018,NaN,SI


In [7]:
len(servir_data["CENT_RUC"].unique())

1110

In [8]:
servir_data.columns

Index(['NTDI_CODIGO', 'CTDI_DESCRIPCION', 'CSAS_PER_NRODOC', 'CENT_RUC',
       'CENT_NOMBRE', 'OBANT_NIVEL_GOB', 'OBANT_SECTOR', 'NOMBRE_DEPARTAMENTO',
       'NOMBRE_PROVINCIA', 'NOMBRE_DISTRITO', 'CSAS_CATEGORIA', 'CSAS_TIPO',
       'CSAS_CAUSA', 'DSAS_FECINI', 'DSAS_FECFIN', 'CSAS_INHABILITA'],
      dtype='object')

In [13]:
servir_data["CSAS_TIPO"].unique()

array(['PRIVACIÓN DE LA FUNCIÓN, CARGO O COMISIÓN QUE EJERCÍA EL CONDENADO, AUNQUE PROVENGA DE ELECCIÓN POPULAR',
       'DESTITUCION - LEY 29988', 'CESE TEMPORAL', 'SUSPENSIÓN',
       'INHABILITACION DEFINITIVA - LEY 29988', 'INHABILITACIÓN',
       'CONDENAS PENALES POR DELITOS CONTRA LA ADMINISTRACIÓN PUBLICA',
       'CONDENA PENAL POR DELITOS ESTABLECIDOS EN LA LEY 29988',
       'DESTITUCIÓN',
       'INCAPACIDAD DEFINITIVA PARA INGRESAR O REINGRESAR AL SERVICIO DOCENTE O ADMINISTRATIVO EN INSTITUCIONES DE EDUCACIÓN […]',
       'INHABILITACION EX SERVIDORES',
       'INCAPACIDAD PARA OBTENER MANDATO, CARGO, EMPLEO O COMISIÓN DE CARÁCTER PÚBLICO',
       'DESPIDO', 'INHABILITACION', 'RESOLUCION CONTRACTUAL - LEY 29988',
       'DESPIDO - LEY 29988', 'SUSPENSIÓN TEMPORAL',
       'CESE TEMPORAL - LEY 30220', 'MULTA - RA 227-2009-PJ'],
      dtype=object)